# P4Lang Tutorials of FABRIC

This notebook walks the user through setting up a FABRIC eperiment that is suitiable for completing the P4 tutorials created by [P4Lang](https://github.com/p4lang/tutorials). The tutorials were origianlly designed to use a mininet topology. This example replaces the mininet topology with a FABRIC experiemnt topology that may span multiple sites across the FABRIC testbed.

Additional resources:
- [FABRIC Knowledge Base](https://learn.fabric-testbed.net/)
- [FABRIC Forums](https://learn.fabric-testbed.net/forums/)
- [P4Lang Tutorials](https://github.com/p4lang/tutorials)
- [P4Lang YouTube Presentations](https://www.youtube.com/channel/UCOQAFkDKucJWr-KafdJsdIQ)

## Basic FABRIC Slice Configuration

In [3]:
import json
import traceback
from fabrictestbed_extensions.fablib.fablib import fablib

In [4]:
try:
    print(f"{fablib.list_sites()}")
except Exception as e:
    print(f"Exception: {e}")

Name,Address,Location,Hosts,CPUs,Cores Available,Cores Capacity,Cores Allocated,RAM Available,RAM Capacity,RAM Allocated,Disk Available,Disk Capacity,Disk Allocated,Basic NIC Available,Basic NIC Capacity,Basic NIC Allocated,ConnectX-6 Available,ConnectX-6 Capacity,ConnectX-6 Allocated,ConnectX-5 Available,ConnectX-5 Capacity,ConnectX-5 Allocated,NVMe Available,NVMe Capacity,NVMe Allocated,Tesla T4 Available,Tesla T4 Capacity,Tesla T4 Allocated,RTX6000 Available,RTX6000 Capacity,RTX6000 Allocated
GATECH,"760 West Peachtree Street NW Atlanta, GA 30308","(33.7753991, -84.3875488)",5,10,318,320,2,2552,2560,8,116390,116400,10,634,635,1,2,2,0,4,4,0,16,16,0,0,0,0,0,0,0
UCSD,"10100 Hopkins Dr, San Diego, CA 92121","(32.8881832, -117.2388161)",5,10,258,320,62,2088,2560,472,113030,116400,3370,611,635,24,2,2,0,4,4,0,16,16,0,4,4,0,6,6,0
GPN,"5115 Oak Street, Kansas City, MO 64112","(39.03426274760282, -94.58260749540294)",5,10,320,320,0,2560,2560,0,116400,116400,0,635,635,0,2,2,0,4,4,0,16,16,0,4,4,0,6,6,0
NCSA,"1725 S Oak St.,Champaign, IL 61820","(40.1035624, -88.2415105)",3,6,98,192,94,1256,1536,280,58280,60600,2320,365,381,16,0,2,2,2,2,0,10,10,0,2,2,0,3,3,0
CLEM,"340 Computer Court,Anderson, SC 29625","(34.586543500000005, -82.82128891709674)",3,6,102,192,90,1368,1536,168,60290,60600,310,377,381,4,2,2,0,0,2,2,10,10,0,2,2,0,3,3,0
MICH,"2530 Draper Dr,Ann Arbor, MI 48109","(42.2931086, -83.7101319)",3,6,172,192,20,1488,1536,48,60480,60600,120,377,381,4,2,2,0,2,2,0,10,10,0,2,2,0,3,3,0
FIU,"11001 SW 14th St,Miami, FL 33199","(25.754495891386522, -80.37232833001887)",5,10,262,320,58,2360,2560,200,114490,116400,1910,624,635,11,2,2,0,4,4,0,16,16,0,4,4,0,6,6,0
STAR,"710 North Lake Shore Dr,Chicago, IL 60611","(41.89537135, -87.61663220067463)",6,12,334,384,50,2876,3072,196,120610,121200,590,751,762,11,2,2,0,6,6,0,13,20,7,6,6,0,6,6,0
SALT,"572 Delong St,Salt Lake City, UT 84104","(40.75707505789612, -111.95346637770317)",3,6,152,192,40,1424,1536,112,60480,60600,120,375,381,6,2,2,0,2,2,0,10,10,0,2,2,0,3,3,0
UTAH,"875 South West Temple,Salt Lake City, UT 84101","(40.760854, -111.8939479)",5,10,212,320,108,2240,2560,320,113700,116400,2700,617,635,18,2,2,0,4,4,0,16,16,0,4,4,0,5,5,0


## Configure Slice Parameters

This section builds the experiment slice 

<img src="figs/fabric_slice.png" width="800"/>



In [8]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

# Slice 
slice_name = 'DDOSM-18'

[site1,site2,site3] = fablib.get_random_sites(count=3)
site1 = site2 = site3 = "TACC" # I was only able to get this working with all nodes on the same site currently
print(f"Sites: {site1},{site2},{site3}")

# Switches
s1_name = "s1"

switch_cores = 8
switch_ram = 16
switch_disk = 120

# Hosts
h1_name = "h1"
h2_name = "h2"
h3_name = "h3"
h4_name = "h4"

h1_subnet=IPv4Network('10.0.1.0/24')
h1_addr=IPv4Address('10.0.1.1')

h2_subnet=IPv4Network('10.0.2.0/24')
h2_addr=IPv4Address('10.0.2.2')

h3_subnet=IPv4Network('10.0.3.0/24')
h3_addr=IPv4Address('10.0.3.3')

h4_subnet=IPv4Network('10.0.4.0/24')
h4_addr=IPv4Address('10.0.4.4')

host_cores = 8
host_ram = 16
host_disk = 120
server_disk = 100

net_h1_name = 'net_h1'
net_h2_name = 'net_h2'
net_h3_name = 'net_h3'
net_h4_name = 'net_h4'

# All node properties
image = 'default_ubuntu_18'

Sites: TACC,TACC,TACC


### Create the Slice

In [9]:
try:
    #Create Slice
    slice = fablib.new_slice(name=slice_name)
    
    # Add switch node s1
    s1 = slice.add_node(name=s1_name, site=site1,  image=image, 
                        cores=switch_cores, ram=switch_ram, disk=switch_disk)
    s1.set_capacities(cores=switch_cores, ram=switch_ram, disk=switch_disk)
    s1_iface_to_h1 = s1.add_component(model='NIC_Basic', name="s1_host1_nic").get_interfaces()[0]
    s1_iface_to_h2 = s1.add_component(model='NIC_Basic', name="s1_host2_nic").get_interfaces()[0]
    s1_iface_to_h3 = s1.add_component(model='NIC_Basic', name="s1_host3_nic").get_interfaces()[0]
    s1_iface_to_h4 = s1.add_component(model='NIC_Basic', name="s1_host4_nic").get_interfaces()[0]
    
    # Add host node h1
    h1 = slice.add_node(name=h1_name, site=site1, image=image,
                        cores=host_cores, ram=host_ram, disk=host_disk)
    h1_iface = h1.add_component(model='NIC_Basic', name="h1_nic").get_interfaces()[0]
    
    # Add host node h2
    h2 = slice.add_node(name=h2_name, site=site2, image=image,
                        cores=host_cores, ram=host_ram, disk=host_disk)
    h2_iface = h2.add_component(model='NIC_Basic', name="h2_nic").get_interfaces()[0]
    
    # Add host node h3
    h3 = slice.add_node(name=h3_name, site=site3, image=image,
                        cores=host_cores, ram=host_ram, disk=host_disk)
    h3_iface = h3.add_component(model='NIC_Basic', name="h3_nic").get_interfaces()[0]
    
    # Add host node h4
    h4 = slice.add_node(name=h4_name, site=site1, image=image,
                        cores=host_cores, ram=host_ram, disk=host_disk)
    h4_iface = h4.add_component(model='NIC_Basic', name="h4_nic").get_interfaces()[0]

    #Add switch-host links
    host_net1 = slice.add_l2network(name=net_h1_name, interfaces=[s1_iface_to_h1, h1_iface])
    host_net2 = slice.add_l2network(name=net_h2_name, interfaces=[s1_iface_to_h2, h2_iface])
    host_net3 = slice.add_l2network(name=net_h3_name, interfaces=[s1_iface_to_h3, h3_iface])
    host_net4 = slice.add_l2network(name=net_h4_name, interfaces=[s1_iface_to_h4, h4_iface])

    #Submit Slice Request
    slice.submit() 
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()



Retry: 9, Time: 357 sec


ID,17222380-c6d2-44f0-836e-8b6b6f07f0b4
Name,DDOSM-18
Lease Expiration (UTC),2023-05-03 03:33:42 +0000
Lease Start (UTC),2023-05-02 03:33:43 +0000
Project ID,6ce270de-788d-4e07-8bae-3206860a6387
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
f03413bd-b5eb-40e2-a316-454a4a893c81,s1,8,16,500,default_ubuntu_18,qcow2,tacc-w1.fabric-testbed.net,TACC,ubuntu,129.114.110.113,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@129.114.110.113,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
23e7c941-5a3c-4a1c-b39b-813bc4756cb2,h1,8,16,500,default_ubuntu_18,qcow2,tacc-w1.fabric-testbed.net,TACC,ubuntu,129.114.110.90,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@129.114.110.90,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
818c173e-e37a-42a0-9891-bd6e612ba7ea,h2,8,16,500,default_ubuntu_18,qcow2,tacc-w1.fabric-testbed.net,TACC,ubuntu,129.114.110.81,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@129.114.110.81,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
420ccc45-5011-4632-87e8-77cb61489983,h3,8,16,500,default_ubuntu_18,qcow2,tacc-w1.fabric-testbed.net,TACC,ubuntu,129.114.110.79,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@129.114.110.79,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
82f457f7-36d0-41b2-94d5-6192f22829dd,h4,8,16,500,default_ubuntu_18,qcow2,tacc-w1.fabric-testbed.net,TACC,ubuntu,129.114.110.120,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@129.114.110.120,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Gateway,Subnet,State,Error
c5664e2b-82a0-4721-88d3-4d7f877b7f52,net_h1,L2,L2Bridge,TACC,None,None,Active,
fcce5f84-89ac-4151-8d15-addc64906fc1,net_h2,L2,L2Bridge,TACC,None,None,Active,
93199cb7-2c0c-4a07-b294-01755ad76213,net_h3,L2,L2Bridge,TACC,None,None,Active,
b054713c-0772-45e2-95a6-67f6bdc6511c,net_h4,L2,L2Bridge,TACC,None,None,Active,



Time to stable 357 seconds
Running post_boot_config ... Time to post boot config 384 seconds


Name,Node,Network,Bandwidth,VLAN,MAC,Physical Device,Device
s1-s1_host1_nic-p1,s1,net_h1,100,None,0E:39:E8:53:0D:F9,ens8,ens8
s1-s1_host4_nic-p1,s1,net_h4,100,None,0A:32:91:06:42:C2,ens7,ens7
s1-s1_host2_nic-p1,s1,net_h2,100,None,0E:BD:97:98:24:91,ens9,ens9
s1-s1_host3_nic-p1,s1,net_h3,100,None,0E:C2:EA:42:B1:E5,ens10,ens10
h1-h1_nic-p1,h1,net_h1,100,None,02:F3:96:01:C7:5E,ens7,ens7
h2-h2_nic-p1,h2,net_h2,100,None,12:02:02:5A:5E:C2,ens7,ens7
h3-h3_nic-p1,h3,net_h3,100,None,12:2E:F1:97:38:02,ens7,ens7
h4-h4_nic-p1,h4,net_h4,100,None,12:0E:02:F2:93:7E,ens7,ens7



Time to print interfaces 399 seconds


## Observe the Slice's Attributes

### Print the slice

In [10]:
try:
    slice = fablib.get_slice(name=slice_name)
    print(f"{slice}")
except Exception as e:
    print(f"Exception: {e}")

-----------  ------------------------------------
Slice Name   DDOSM-P4
Slice ID     4ea6203c-e32a-45df-8e78-d1b78828c363
Slice State  StableOK
Lease End    2023-05-02 07:55:18 +0000
-----------  ------------------------------------


### Print the Node List

In [11]:
try:
    slice = fablib.get_slice(name=slice_name)

    print(f"{slice.list_nodes()}")
except Exception as e:
    print(f"Exception: {e}")

ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
e93277ed-9be6-4e93-a383-2c58c686c2bf,s1,8,16,500,default_ubuntu_20,qcow2,tacc-w2.fabric-testbed.net,TACC,ubuntu,129.114.110.102,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@129.114.110.102,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
ce5a2088-fc57-4a5b-ad96-9872772cd531,h1,8,16,500,default_ubuntu_20,qcow2,tacc-w2.fabric-testbed.net,TACC,ubuntu,129.114.110.80,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@129.114.110.80,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
618e3753-ccaa-453f-b24b-1435a354c07e,h2,8,16,500,default_ubuntu_20,qcow2,tacc-w1.fabric-testbed.net,TACC,ubuntu,129.114.110.84,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@129.114.110.84,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
f0db1523-bc1e-464a-b71e-138b4141093a,h3,8,16,500,default_ubuntu_20,qcow2,tacc-w2.fabric-testbed.net,TACC,ubuntu,129.114.110.85,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@129.114.110.85,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
a1863b5d-c913-4452-b2b6-2a6f21c3d0a9,h4,8,16,500,default_ubuntu_20,qcow2,tacc-w2.fabric-testbed.net,TACC,ubuntu,129.114.110.110,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@129.114.110.110,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


### Print the Node Details

In [12]:
try:
    slice = fablib.get_slice(name=slice_name)
    for node in slice.get_nodes():
        print(f"{node}")
except Exception as e:
    print(f"Exception: {e}")

-----------------  ---------------------------------------------------------------------------------------------------------------------
ID                 e93277ed-9be6-4e93-a383-2c58c686c2bf
Name               s1
Cores              8
RAM                16
Disk               500
Image              default_ubuntu_20
Image Type         qcow2
Host               tacc-w2.fabric-testbed.net
Site               TACC
Management IP      129.114.110.102
Reservation State  Active
Error Message
SSH Command        ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@129.114.110.102
-----------------  ---------------------------------------------------------------------------------------------------------------------
-----------------  --------------------------------------------------------------------------------------------------------------------
ID                 ce5a2088-fc57-4a5b-ad96-9872772cd531
Name               h1
Cores              8
RAM

### Print the Node SSH Commands

In [13]:
try:
    slice = fablib.get_slice(name=slice_name)
    for node in slice.get_nodes():
        print(f"{node.get_name()}: {node.get_ssh_command()}")
except Exception as e:
    print(f"Exception: {e}")

s1: ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@129.114.110.102
h1: ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@129.114.110.80
h2: ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@129.114.110.84
h3: ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@129.114.110.85
h4: ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@129.114.110.110


### Print the Interfaces

In [14]:
try:
    slice = fablib.get_slice(name=slice_name)
    
    print(f"{slice.list_interfaces()}")
except Exception as e:
    print(f"Exception: {e}")

Name,Node,Network,Bandwidth,VLAN,MAC,Physical Device,Device
s1-s1_host1_nic-p1,s1,net_h1,100,None,0A:03:0A:F5:B1:53,None,None
s1-s1_host4_nic-p1,s1,net_h4,100,None,06:A8:0F:D7:32:FB,ens9,ens9
s1-s1_host2_nic-p1,s1,net_h2,100,None,02:7F:22:D0:3A:53,None,None
s1-s1_host3_nic-p1,s1,net_h3,100,None,02:EE:00:C6:75:96,None,None
h1-h1_nic-p1,h1,net_h1,100,None,0A:7C:D4:F9:5A:E4,ens7,ens7
h2-h2_nic-p1,h2,net_h2,100,None,02:26:69:8A:CD:1F,ens7,ens7
h3-h3_nic-p1,h3,net_h3,100,None,0A:A6:17:C6:65:D0,ens7,ens7
h4-h4_nic-p1,h4,net_h4,100,None,02:2A:1F:68:A6:C7,ens7,ens7


## Configure Nodes


In [53]:
config_threads = {}

In [54]:
host_config_script = "sudo apt-get update -qq && sudo apt-get install -qq -y python3-scapy && git clone https://github.com/p4lang/tutorials.git/" 

try:
    
    
    h1 = slice.get_node(name=h1_name)        
    if type(ip_address(h1.get_management_ip())) is IPv6Address:
        h1.execute("sudo sed -i '/nameserver/d' /etc/resolv.conf && sudo sh -c 'echo nameserver 2a00:1098:2c::1 >> /etc/resolv.conf' && sudo sh -c 'echo nameserver 2a01:4f8:c2c:123f::1 >> /etc/resolv.conf' && sudo sh -c 'echo nameserver 2a00:1098:2b::1 >> /etc/resolv.conf'")
       
    h1_os_iface = h1.get_interface(network_name=net_h1_name)
    h1_os_iface.ip_addr_add(addr=h1_addr, subnet=h1_subnet)
    h1_config_thread = h1.execute_thread(host_config_script)
    config_threads[h1] = h1_config_thread

    
    h2 = slice.get_node(name=h2_name)
    if type(ip_address(h2.get_management_ip())) is IPv6Address:
        h2.execute("sudo sed -i '/nameserver/d' /etc/resolv.conf && sudo sh -c 'echo nameserver 2a00:1098:2c::1 >> /etc/resolv.conf' && sudo sh -c 'echo nameserver 2a01:4f8:c2c:123f::1 >> /etc/resolv.conf' && sudo sh -c 'echo nameserver 2a00:1098:2b::1 >> /etc/resolv.conf'")
 
    h2_os_iface = h2.get_interface(network_name=net_h2_name)
    h2_os_iface.ip_addr_add(addr=h2_addr, subnet=h2_subnet)
    h2_config_thread = h2.execute_thread(host_config_script)
    config_threads[h2] = h2_config_thread

    h3 = slice.get_node(name=h3_name)
    if type(ip_address(h3.get_management_ip())) is IPv6Address:
        h3.execute("sudo sed -i '/nameserver/d' /etc/resolv.conf && sudo sh -c 'echo nameserver 2a00:1098:2c::1 >> /etc/resolv.conf' && sudo sh -c 'echo nameserver 2a01:4f8:c2c:123f::1 >> /etc/resolv.conf' && sudo sh -c 'echo nameserver 2a00:1098:2b::1 >> /etc/resolv.conf'")
 
    h3_os_iface = h3.get_interface(network_name=net_h3_name)
    h3_os_iface.ip_addr_add(addr=h3_addr, subnet=h3_subnet)
    h3_config_thread = h3.execute_thread(host_config_script)
    config_threads[h3] = h3_config_thread
    
    h4 = slice.get_node(name=h4_name)
    if type(ip_address(h4.get_management_ip())) is IPv6Address:
        h4.execute("sudo sed -i '/nameserver/d' /etc/resolv.conf && sudo sh -c 'echo nameserver 2a00:1098:2c::1 >> /etc/resolv.conf' && sudo sh -c 'echo nameserver 2a01:4f8:c2c:123f::1 >> /etc/resolv.conf' && sudo sh -c 'echo nameserver 2a00:1098:2b::1 >> /etc/resolv.conf'")

    h4_os_iface = h4.get_interface(network_name=net_h4_name)
    h4_os_iface.ip_addr_add(addr=h4_addr, subnet=h4_subnet)
    h4_config_thread = h4.execute_thread(host_config_script)
    config_threads[h4] = h4_config_thread


except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()

## Configure Switches

Use ssh to configure the ifaces on the switches. This step requires testing the interfaces to figure out which interface is connected to which network.


#### Setup P4 Docker



Below are commands to let sudo work with the global proxy.

In [18]:
s1 = slice.get_node(name=s1_name)
s1.upload_file('CSE545-AWS.pem','CSE545-AWS.pem')

<SFTPAttributes: [ size=0 uid=1000 gid=1000 mode=0o100664 atime=1682993793 mtime=1682993793 ]>

In [55]:
try:
    s1 = slice.get_node(name=s1_name)
    s1_h1_os_iface = s1.get_interface(network_name=net_h1_name)
    s1_h2_os_iface = s1.get_interface(network_name=net_h2_name)
    s1_h3_os_iface = s1.get_interface(network_name=net_h3_name)
    s1_h4_os_iface = s1.get_interface(network_name=net_h4_name)
        
    if type(ip_address(s1.get_management_ip())) is IPv4Address:
        management_ip_type = "IPv4"
    else:
        management_ip_type = "IPv6"
    
    file_attributes = s1.upload_file('./scripts/router_setup_p4_bmv2_container.sh','router_setup_p4_bmv2_container.sh')
    command=f"chmod +x router_setup_p4_bmv2_container.sh && sudo sh -c './router_setup_p4_bmv2_container.sh  {management_ip_type} {s1_h1_os_iface.get_os_interface()} {s1_h2_os_iface.get_os_interface()} {s1_h3_os_iface.get_os_interface()} {s1_h4_os_iface.get_os_interface()} > /tmp/script.log 2>&1'"
    s1_config_thread = s1.execute_thread(command)
    config_threads[s1] = s1_config_thread


except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()

In [56]:
try:
    for node, thread in config_threads.items():
        stdout, stderr = thread.result()
        print(f"Config thread node {node.get_name()} complete")
        print(f"stdout: {stdout}")
        print(f"stderr: {stderr}")
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc() 


Config thread node h1 complete
stdout: Selecting previously unselected package python3-backcall.
(Reading database ... 63577 files and directories currently installed.)
Preparing to unpack .../00-python3-backcall_0.1.0-2_all.deb ...
Unpacking python3-backcall (0.1.0-2) ...
Selecting previously unselected package python3-decorator.
Preparing to unpack .../01-python3-decorator_4.4.2-0ubuntu1_all.deb ...
Unpacking python3-decorator (4.4.2-0ubuntu1) ...
Selecting previously unselected package python3-parso.
Preparing to unpack .../02-python3-parso_0.5.2-1ubuntu1_all.deb ...
Unpacking python3-parso (0.5.2-1ubuntu1) ...
Selecting previously unselected package python3-jedi.
Preparing to unpack .../03-python3-jedi_0.15.2-1_all.deb ...
Unpacking python3-jedi (0.15.2-1) ...
Selecting previously unselected package python3-pickleshare.
Preparing to unpack .../04-python3-pickleshare_0.7.5-2_all.deb ...
Unpacking python3-pickleshare (0.7.5-2) ...
Selecting previously unselected package python3-wcwid

### Confgure P4 Switch Tables

Edit sX_commands.txt to change the values

In [58]:
switch_name = s1_name
switch_node = slice.get_node(name=switch_name)
management_ip_switch = str(switch_node.get_management_ip())
print("Swtitch Name        : {}".format(switch_node.get_name()))
print("Management IP    : {}".format(management_ip_switch))

#Configure P4 Tables
cmd_file=f'{switch_name}_commands.txt'
print(cmd_file)
file_attributes = switch_node.upload_file(f'scripts/{cmd_file}',cmd_file)
print("file_attributes: {}".format(file_attributes))

stdout = switch_node.execute(f"sudo sh -c 'cat {cmd_file} | docker exec -i fabric_p4 simple_switch_CLI'")
print("stdout: {}".format(stdout))

Swtitch Name        : s1
Management IP    : 129.114.110.102
s1_commands.txt
file_attributes: -rw-rw-r--   1 1000     1000          234 01 May 07:10 ?
Obtaining JSON from switch...
Done
Control utility for runtime P4 table manipulation
RuntimeCmd: Setting default action of myTunnel_exact
action:              drop
runtime data:        
RuntimeCmd: Adding entry to exact match table myTunnel_exact
match key:           EXACT-00:01
action:              myTunnel_forward
runtime data:        00:01
Invalid table operation (DUPLICATE_ENTRY)
RuntimeCmd: Adding entry to exact match table myTunnel_exact
match key:           EXACT-00:02
action:              myTunnel_forward
runtime data:        00:02
Invalid table operation (DUPLICATE_ENTRY)
RuntimeCmd: Adding entry to exact match table myTunnel_exact
match key:           EXACT-00:03
action:              myTunnel_forward
runtime data:        00:03
Invalid table operation (DUPLICATE_ENTRY)
RuntimeCmd: Adding entry to exact match table myTunnel_exact


## The switches are now configured and running. Now we are going to send packets over the switches.

We are going to use `send.py` and `receive.py`. We are going to re-upload them to the servers and use them. Make sure to modify the interface names in the script accordingly.

In [67]:
h1.upload_file('scripts/send.py', 'tutorials/exercises/basic_tunnel/send_modified.py')
h4.upload_file('scripts/receive.py', 'tutorials/exercises/basic_tunnel/receive_modified.py')

#h1.execute('sudo mv send.py tutorials/exercises/basic_tunnel/send_modified.py')
#h2.execute('sudo mv receive.py tutorials/exercises/basic_tunnel/receive_modified.py')

<SFTPAttributes: [ size=882 uid=1000 gid=1000 mode=0o100664 atime=1682925152 mtime=1682925152 ]>

## Below, we send a few packets.

In [70]:
h1.execute('tmux new -d \'timeout 30 watch -n 5 sudo python3 tutorials/exercises/basic_tunnel/send_modified.py --dst_id 2 10.10.4.4 "message100"\'')

('', '')

## And below we receive them.

In [71]:
print(h4.execute('sudo timeout 30 sudo python3 tutorials/exercises/basic_tunnel/receive_modified.py')[0])

['ens7', 'ens3']
sniffing on ens3
got a packet
###[ Ethernet ]### 
  dst       = ff:ff:ff:ff:ff:ff
  src       = fa:16:3e:52:f8:0c
  type      = 0x1212
###[ MyTunnel ]### 
     pid       = 2048
     dst_id    = 2
###[ IP ]### 
        version   = 4
        ihl       = 5
        tos       = 0x0
        len       = 30
        id        = 1
        flags     = 
        frag      = 0
        ttl       = 64
        proto     = hopopt
        chksum    = 0x5cd8
        src       = 10.20.5.230
        dst       = 10.10.4.4
        \options   \
###[ Raw ]### 
           load      = 'message100'

got a packet
###[ Ethernet ]### 
  dst       = ff:ff:ff:ff:ff:ff
  src       = fa:16:3e:52:f8:0c
  type      = 0x1212
###[ MyTunnel ]### 
     pid       = 2048
     dst_id    = 2
###[ IP ]### 
        version   = 4
        ihl       = 5
        tos       = 0x0
        len       = 30
        id        = 1
        flags     = 
        frag      = 0
        ttl       = 64
        proto     = hopopt
      

 ## Delete Slice

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    slice.delete()
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()

## Get and Renew the Slice

You slice is in the list of all your slices. You can loop through the list of slices to get the slice. Python has a standard tool to filter lists. Try using a lambda function to filter out your slice using its name.

The new end date is in UTC.

In [ ]:
from datetime import datetime
from datetime import timezone
from datetime import timedelta

#Set end host to now plus 1 day
end_date = (datetime.now(timezone.utc) + timedelta(days=1)).strftime("%Y-%m-%d %H:%M:%S %z")

try:
    slice = fablib.get_slice(name=slice_name)

    slice.renew(end_date)
except Exception as e:
    print(f"Exception: {e}")

## Check New Lease End Date

In [76]:
try:
    slice = fablib.get_slice(name=slice_name)
    print(f"Lease End (UTC)        : {slice.get_lease_end()}")
       
except Exception as e:
    print(f"Exception: {e}")

Lease End (UTC)        : 2023-05-02 07:55:18 +0000
